In [1]:
import torch
from torch.autograd import Variable # Deprecated -> Torch에 지원
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset # DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

## 1. Wine 데이터

### 데이터 준비

In [2]:
wine = load_wine()
wine.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [3]:
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0


In [4]:
df.shape, wine.target_names, wine.target

((178, 13),
 array(['class_0', 'class_1', 'class_2'], dtype='<U7'),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2]))

In [5]:
wine_data = wine.data[:130]
wine_target = wine.target[:130]
wine_data, wine_target

(array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.179e+01, 2.130e+00, 2.780e+00, ..., 9.700e-01, 2.440e+00,
         4.660e+02],
        [1.237e+01, 1.630e+00, 2.300e+00, ..., 8.900e-01, 2.780e+00,
         3.420e+02],
        [1.204e+01, 4.300e+00, 2.380e+00, ..., 7.900e-01, 2.570e+00,
         5.800e+02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [24]:
train_X, test_X, train_y, test_y = train_test_split(wine_data, wine_target, test_size=.2)

In [25]:
print(train_X.shape)
print(test_X.shape)

(104, 13)
(26, 13)


### 텐서 생성

In [26]:
# This is the Code
# tensor from numpy
train_X = torch.from_numpy(train_X).float() # double(float64)->float(float32)
train_y = torch.from_numpy(train_y).long() # long -> long (int64)

In [27]:
test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y).long()

In [28]:
print(train_X.size()) # ->104행 13열 (2차원배열)
print(train_y.size()) # ->104행 1열  (1차원배열)

torch.Size([104, 13])
torch.Size([104])


In [29]:
# This is the Code
train = TensorDataset(train_X, train_y) # like Dataframe

In [30]:
train

In [31]:
train.tensors

(tensor([[1.2080e+01, 1.8300e+00, 2.3200e+00,  ..., 1.0800e+00, 2.2700e+00,
          4.8000e+02],
         [1.2080e+01, 2.0800e+00, 1.7000e+00,  ..., 1.2700e+00, 2.9600e+00,
          7.1000e+02],
         [1.2330e+01, 1.1000e+00, 2.2800e+00,  ..., 1.2500e+00, 1.6700e+00,
          6.8000e+02],
         ...,
         [1.2340e+01, 2.4500e+00, 2.4600e+00,  ..., 8.0000e-01, 3.3800e+00,
          4.3800e+02],
         [1.3770e+01, 1.9000e+00, 2.6800e+00,  ..., 1.1300e+00, 2.9300e+00,
          1.3750e+03],
         [1.3760e+01, 1.5300e+00, 2.7000e+00,  ..., 1.2500e+00, 3.0000e+00,
          1.2350e+03]]),
 tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
         1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
         0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
         1, 0, 0, 0, 0, 1, 0, 0]))

In [32]:
print("train_X", train.tensors[0].size())
print("train_y", train.tensors[1].size())

train_X torch.Size([104, 13])
train_y torch.Size([104])


In [33]:
# This is the Code
train_loader = DataLoader(train, batch_size=16, shuffle=True) 
train_loader

### 신경망 구성 - P.79

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

In [35]:
# model Instance 생성
model = Net()

### 모형 학습 - P.81

In [36]:
# Cross Entropy Loss(Cost) Function
criterion = nn.CrossEntropyLoss()
# metric = nn.CrossEntropyLoss()

In [37]:
# Stochastic Gradient Decent
optimizer = optim.SGD(model.parameters(), lr=0.01) # learning late

In [38]:
for epoch in range(300):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        # train_x, train_y = Variable(train_x), Variable(train_y) # Deprecated
        # train_x.requires_grad_()
        
        # RuntimeError: only Tensors of floating point dtype can require gradients
        # train_y = train_y.float()
        # train_y.requires_grad_()
        # train_y = Variable(train_y)
        
        optimizer.zero_grad() # reset gradient
        
        output = model(train_x)
        
        loss = criterion(output, train_y)

        loss.backward()
        
        optimizer.step()
        
        # total_loss += loss.data[0]
        total_loss += loss.item()
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)
        print("="*40)

50 0.9416836798191071
100 1.310624647885561
150 0.9749254956841469
200 0.7650846838951111
250 0.871158242225647
300 0.9092356041073799


In [39]:
type(test_X)

torch.Tensor

In [40]:
# 출력이 0 혹은 1이 되게 함
# result = torch.max(model(test_X), 1)[1]
result = torch.max(model(test_X), 1).indices

#정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

AttributeError: 'tuple' object has no attribute 'indices'